# Your name and your onyen

Classmates you worked with today:

- name
- name
- name
  
**Submission instruction**: Please create a pdf via File -> Print (or cmd + P on mac), and upload it to Gradescope. No autograder since there are many correct ways to approach this question. You are not required to finish the entire worksheet provided that you were actively engaged during the entire class period. As long as you've collaborated, put in good effort and made reasonable progress during the lab period, you can expect to get full credit. So even if you decide to finish it at home, please submit what you have by the end of 50 minutes to make sure you get credit!

## Part A: Are we automating racism? Video by Vox (~25 minutes)

The whole class will watch this video together:

https://www.youtube.com/watch?v=Ok5sKLXqynQ

By the way, the paper discussed around minute 14 is where project C2 comes from.

While you're watching the video, answer this question: What are two new or interesting things you learned from this video?

[Your answer]

### Discussion as a group (5 minutes)

What is one thing you learned or find interesting from your classmates' answers?

[Your answer]

## Part B: Linear regression (20 minutes)

Let's revisit the ROUSes dataset. 

In [2]:
import pandas as pd
import seaborn as sns

In [3]:
rouses = pd.read_csv('ROUSes.csv')
print(rouses.shape)
rouses.head()

(29, 4)


,Age,Length,Weight,Temperament
0,9.5,4.1,93.2,Sleepy
1,12.0,4.1,97.7,Moody
2,14.5,4.6,120.3,No-nonsense
3,7.0,3.3,60.4,Moody
4,10.0,3.5,75.3,No-nonsense


Run the following code to drop the column `Temperament` from the table `rouses`. 

In [4]:
rouses = rouses.drop(columns='Temperament')

Here our goal is to predict the `Weight` of ROUSes using numerical values. Start by answering the following questions:

- Classification or regression?
- Predictor variables?
- Target variables?
- Model: is this simiple or multiple linear regression?

[Your answer]

### Exploratory analysis

Run the code `rouses.corr()`. This calculates the correlations between pairs of numerical columns.

In [ ]:
# write your code here



Based on the correlations, which variable do you expect to be more helpful in predicting `Weight`: `Age` or `Length`?

[Your answer]

### Setting up the data

We usually split the dataset into train and test first and then split them into X and y. But for this exercise, let's go in a different order.

Split the table `rouses` into `X` and `y` by completing the following code.

```Python
y = rouses[...]
X = rouses.drop(columns=...)
```

In [ ]:
# Write your code here



### Setting up regression pipeline

We'll write our machine learning code inside a function so that we can call it multiple times later. Copy the following code, then fill in the missing spots marked by `...`.

**Hint**: The six `...` are from these options: `X_train`, `X_test`, `y_train`, `y_test`.

``` Python
def rouses_lr(X, y, seed=0):
    # Split X and y into X_train, X_test and y_train, y_test
    # using the same seed ensures that the same rows are picked between X and y
    
    X_train = X.sample(frac=0.8, random_state=seed)
    X_test = X.drop(index=X_train.index) 
    
    y_train = y.sample(frac=0.8, random_state=seed)
    y_test = y.drop(index=y_train.index)
    
    # Create "empty" model
    from sklearn.linear_model import LinearRegression
    
    lr = LinearRegression(fit_intercept=True)
    
    # Fit model to data (or train model)
    lr.fit(..., ...)
    
    # Save coefficients of the trained model
    coefs = pd.DataFrame(lr.coef_, 
                         index=lr.feature_names_in_, 
                        columns=['Coefficient vals'])
    
    # Save model performance on train and test
    coefs.loc['Train R2 score'] = lr.score(..., ...)
    coefs.loc['Test R2 score'] = lr.score(..., ...)
    return coefs
``` 


In [ ]:
# complete and run your code here



### Normalizing features

In class, we talked about interpreting linear regression coefficients, and mentioned an important caviat: To use the **magnitude of the coefficients** as an indication of relative feature importance, **the features have to be in the same range or scale**. Today, we'll see a few different ways to acheive that goal. The technical term for this is **feature normalization**, if you want to Google it for your project.

First, call the function `describe` on the table `X` to see the summary statistics of the numerical columns.

In [ ]:
# write your code here



Question: Does `Age` and `Length` have the same ranges? Or the same means?

[Your answer]

We'll see how that affects the final model and interpretation of coefficients.

### 1. Standardization

The first method is called **standardization**. This means that each variable will have mean of 0 and standard deviation of 1 after this process. You can acheive this by doing the following steps:
   1. Within each column, subtract its mean. For example, if the mean `Age` is 13, subtract 13 from everyone's age.
   2. Within each column, divide by its standard deviation. For example, if the std of `Age` 10, divide everyone's age by 10.

The code to achieve this is very simple. Make sure you understand what's going on, then copy and run the lines below.

```Python
X_standardized = X - X.mean() # Subtract by column mean
X_standardized = X_standardized/X_standardized.std() # Divide by column std

X_standardized.describe()
```


In [ ]:
# run your code


Before we move on, check that each feature in `X_standardized` now has zero mean and 1 stadard deviation as expected.

### 2. Min-max scaling

The second method is called **min-max scaling**. This makes all values in each column to fall between 0 and 1. The minimum value becomes 0, the maximum value becomes 1, and everything in between is scaled linearly.

Here is the code that does that. As before, make sure you understand what's going on.

```Python
X_minmax = X - X.min() # this turns minimum value of each column to 0
ranges = X.max() - X.min() # calculate the range per column
X_minmax = X_minmax/ranges # this turns maximum value to 1

X_minmax.describe()
```

In [ ]:
# run your code


Before we move on, check that each feature in `X_minimax` now has minimum 0 and maximum 1 as expected.

### Check the results!

Finally it's time to put everything together. Run the following code to check:

rows: 
- the resulting coefficients (`Age`, `Length`) of the linear regression model, and the
- $R^2$ scores (`Train R2 score`, `Test R2 score`), for

columns:
- the three different versions of feature tables (unnormalized `X`, standardized `X_standardized`, min-max scaled `X_minmax`), and
- the unnormalized coefficients muliplied by the std of the unnormalized features (`Unnormalized * std`)

In [ ]:
seed = 2024

results = pd.DataFrame()
for features, name in zip([X, X_standardized, X_minmax], 
                    ["Unnormalized", "Standardized", "Min-max scaled"]):
    res = rouses_lr(features, y, seed)
    res = res.rename(columns ={'Coefficient vals': name})
    results = pd.concat((results, res), axis=1)

results['Unnormalized * std'] = results['Unnormalized']*X.std()
results

The last column is sometimes called **beta** or **standardized** coefficients. Does it look similar to any of the other columns?

Question: Which feature is more important, according to the magnitude of the coefficients? Did that answer change for different versions of features?

[Your answer]

I hope this convinces you to normalize your features for your project if you decide to do a predictive model, and especially if you plan to do linear regression.

FYI, you get the same score regardless of feature normalization here. This is not the case for every dataset and every ML model. Often normalization improves accuracy of the model.